# GenAI Chatbot For Prehab4Me



## Overview

### Prehab4Me on Gemini 1.5 Pro


### Prehab4Me on Gemini 1.5 Flash



## Getting Started


### Install Vertex AI SDK for Python


In [1]:
%pip install --upgrade --user google-cloud-aiplatform\
                                        gitpython \
                                        magika

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this director

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.


In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "hypnotic-seat-443315-a1"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

### Import libraries


In [3]:
from vertexai.generative_models import GenerativeModel, Image, Part

### Use the Gemini 1.5 models

Gemini 1.5 Pro and Gemini 1.5 Flash are multimodal models that support multimodal prompts. You can include text, image(s), and video in your prompt requests.


In [4]:
multimodal_model = GenerativeModel("gemini-1.5-pro")

multimodal_model_flash = GenerativeModel("gemini-1.5-flash")

### Define helper functions


In [5]:
import http.client
import typing
import urllib.request

from IPython.core.interactiveshell import InteractiveShell
import IPython.display

InteractiveShell.ast_node_interactivity = "all"


def get_image_bytes_from_url(image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
        response = typing.cast(http.client.HTTPResponse, response)
        image_bytes = response.read()
    return image_bytes


def load_image_from_url(image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


def display_content_as_video(content: str | Image | Part):
    if not isinstance(content, Part):
        return False
    part = typing.cast(Part, content)
    file_path = part.file_data.file_uri.removeprefix("gs://")
    video_url = f"https://storage.googleapis.com/{file_path}"
    IPython.display.display(IPython.display.Video(video_url, width=350))

## Combining multiple modalities

### All modalities (images, video, audio, text) at once

Gemini 1.5 Pro is natively multimodal and supports interleaving of data from different modalities. It can support a mix of audio, visual, text, and code inputs in the same input sequence.

In [6]:
!pip install gradio
import gradio as gr

# PDF URLs (fixed internally)
pdf_urls = {
    "Eating Well for Surgery": "https://storage.googleapis.com/prehabformedataweek1/Eating-Well-for-Surgery-Video-Transcription.pdf",
    "Protein Cheat Sheet": "https://storage.googleapis.com/prehabformedataweek1/Protein-Cheat-Sheet.pdf",
    "How to Use a Recipe": "https://storage.googleapis.com/prehabformedataweek1/if-nfs-how-to-use-a-recipe.pdf"
}

# Function to generate a response (internal PDFs + query)
def generate_response(query, history):
    # Combine query with the existing history
    conversation = "\n".join(history) + f"\nUser: {query}"
    conversation += "\nAI:"  # Prompt the model for the next response

    # Use all PDFs internally
    query_context = query + " in the given data"
    contents = [
        Part.from_uri(pdf_urls["Eating Well for Surgery"], mime_type="application/pdf"),
        Part.from_uri(pdf_urls["Protein Cheat Sheet"], mime_type="application/pdf"),
        Part.from_uri(pdf_urls["How to Use a Recipe"], mime_type="application/pdf"),
        query_context
    ]

    # Generate content using the multimodal model
    response = multimodal_model.generate_content(contents)

    # Update history with the new interaction
    history.append(f"User: {query}")
    history.append(f"AI: {response.text}")

    # Return the updated history for display
    return "\n".join(history), history


# Gradio Interface
interface = gr.Interface(
    fn=generate_response,
    inputs=[
        gr.Textbox(label="Enter your query", placeholder="e.g., How to eat well for surgery?"),  # User input
        gr.State([])  # Chat history state
    ],
    outputs=[
        gr.Textbox(label="Chat History", lines=15),  # Display chat history
        gr.State([])  # Updated state
    ],
    title="Prehab4Me GenAi Chatbot",
    description="Ask questions about Eating Well, Protein, or Recipes, and get responses from PDFs, Images & Videos."
)

# Launch the Gradio app
interface.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f25efd4f57be5fb55a.gradio.live

This share link expires in 72 hours

In [7]:
!pip install gradio

import gradio as gr
import json

# PDF URLs (fixed internally)
pdf_urls = {
    "Eating Well for Surgery": "https://storage.googleapis.com/prehabformedataweek1/Eating-Well-for-Surgery-Video-Transcription.pdf",
    "Protein Cheat Sheet": "https://storage.googleapis.com/prehabformedataweek1/Protein-Cheat-Sheet.pdf",
    "How to Use a Recipe": "https://storage.googleapis.com/prehabformedataweek1/if-nfs-how-to-use-a-recipe.pdf"
}

# Feedback storage
feedback_data = []

# Function to generate a response (internal PDFs + query)
def generate_response(query, history):
    # Combine query with the existing history
    conversation = "\n".join(history) + f"\nUser: {query}"
    conversation += "\nAI:"  # Prompt the model for the next response

    # Use all PDFs internally
    query_context = query + " in the given data"
    contents = [
        Part.from_uri(pdf_urls["Eating Well for Surgery"], mime_type="application/pdf"),
        Part.from_uri(pdf_urls["Protein Cheat Sheet"], mime_type="application/pdf"),
        Part.from_uri(pdf_urls["How to Use a Recipe"], mime_type="application/pdf"),
        query_context
    ]

    # Generate content using the multimodal model
    response = multimodal_model.generate_content(contents)

    # Update history with the new interaction
    history.append(f"User: {query}")
    history.append(f"AI: {response.text}")

    # Return the updated history for display
    return "\n".join(history), history, response.text


# Function to collect feedback and save it to JSON
def collect_feedback(response, feedback, history):
    # Append feedback to the list
    feedback_entry = {"response": response, "feedback": feedback, "history": history}
    feedback_data.append(feedback_entry)

    # Save feedback to a JSON file
    with open("feedback_data.json", "w") as json_file:
        json.dump(feedback_data, json_file, indent=4)

    return f"Feedback received: {feedback}"


# Gradio Interface
with gr.Blocks() as demo:
    # Input for user query
    query = gr.Textbox(label="Enter your query", placeholder="e.g., How to eat well for surgery?")

    # Chat history
    chat_history = gr.Textbox(label="Chat History", lines = 5,  interactive=False)
    response_output = gr.Textbox(label="Model Response", interactive=False)

    # Buttons for feedback
    feedback_buttons = gr.Radio(["Like", "Dislike"], label="Feedback on Response")
    feedback_result = gr.Textbox(label="Feedback Status", interactive=False)

    # State for history
    history_state = gr.State([])

    # Buttons for generating a response
    generate_btn = gr.Button("Generate Response")
    feedback_btn = gr.Button("Submit Feedback")

    # Link functionality
    generate_btn.click(
        generate_response,
        inputs=[query, history_state],
        outputs=[chat_history, history_state, response_output]
    )
    feedback_btn.click(
        collect_feedback,
        inputs=[response_output, feedback_buttons, history_state],
        outputs=[feedback_result]
    )
# Launch the app
demo.launch()


{'id': 0,
 'targets': [(25, 'click')],
 'inputs': [19, 24],
 'outputs': [20, 24, 21],
 'backend_fn': True,
 'js': None,
 'queue': True,
 'api_name': 'generate_response',
 'scroll_to_output': False,
 'show_progress': 'full',
 'batch': False,
 'max_batch_size': 4,
 'cancels': [],
 'types': {'generator': False, 'cancel': False},
 'collects_event_data': False,
 'trigger_after': None,
 'trigger_only_on_success': False,
 'trigger_mode': 'once',
 'show_api': True,
 'zerogpu': False,
 'rendered_in': None,
 'connection': 'sse',
 'time_limit': None,
 'stream_every': 0.5,
 'like_user_message': False,
 'event_specific_args': None}

{'id': 1,
 'targets': [(26, 'click')],
 'inputs': [21, 22, 24],
 'outputs': [23],
 'backend_fn': True,
 'js': None,
 'queue': True,
 'api_name': 'collect_feedback',
 'scroll_to_output': False,
 'show_progress': 'full',
 'batch': False,
 'max_batch_size': 4,
 'cancels': [],
 'types': {'generator': False, 'cancel': False},
 'collects_event_data': False,
 'trigger_after': None,
 'trigger_only_on_success': False,
 'trigger_mode': 'once',
 'show_api': True,
 'zerogpu': False,
 'rendered_in': None,
 'connection': 'sse',
 'time_limit': None,
 'stream_every': 0.5,
 'like_user_message': False,
 'event_specific_args': None}

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7772ec2865628c7771.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
